In [6]:
%load_ext autoreload
%autoreload 2

import sys
import logging
from logging import Logger
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
logging.basicConfig(filename = 'scraping.log', level = logging.INFO)
logger = logging.getLogger("main")
logger.info("This is a test message")

# MyRealty

In [ ]:


# MyRealty
from ConcreteScrapers.MyRealty.MyRealtyScrapingPipeline import MyRealtyScrapingPipeline

# Storage
from ConcreteStorages import CSVStorage

# Services
from Services import ImageLoader

storage = CSVStorage(
    file_path = "../data/myrealty_apartments.csv",
    images_path = "../images/",
    logger = logger
)
storage.initialize()

image_loader = ImageLoader(
    storage,
    logger = logger
)

my_realty_scraper_pipeline = MyRealtyScrapingPipeline(
    "https://myrealty.am/en/apartments-for-sale/7784", 
    storage,
    image_loader = image_loader,
    logger = logger
)

links = my_realty_scraper_pipeline.get_apartment_links()
for link in links:
    my_realty_scraper_pipeline.scrape_apartment(link)
    print("Scraped", link)
storage.close_file()

Creating the file: ../data/myrealty_apartments.csv
Scraped https://myrealty.am/en/3-bedroom/apartment-for-sale/Ghazar+Parpetsi+St/Center/Yerevan/142292
Scraped https://myrealty.am/en/3-bedroom/apartment-for-sale-in-new-construction/A.+Khachatrian+St/Arabkir/Yerevan/142291
Scraped https://myrealty.am/en/4-bedroom/apartment-for-sale/Saryan+St/Center/Yerevan/142290
Scraped https://myrealty.am/en/2-bedroom/apartment-for-sale-in-new-construction/Leo+St/Center/Yerevan/142288
Scraped https://myrealty.am/en/3-bedroom/apartment-for-sale-in-new-construction/Tigran+Petrosyan+St/Davtashen/Yerevan/142285


In [24]:
# MyRealty
from ConcreteScrapers.Bars.BarsApartmentScrapingPipeline import BarsApartmentScrapingPipeline

# Storage
from ConcreteStorages import CSVStorage

# Services
from Services import ImageLoader

storage = CSVStorage(
    file_path = "../data/bars_apartments.csv",
    images_path = "../images/",
    logger = logger
)
storage.initialize()

image_loader = ImageLoader(
    storage,
    logger = logger
)

bars_scraper_pipeline = BarsApartmentScrapingPipeline(
    "https://bars.am/en/properties/standard/apartment", 
    storage,
    image_loader,
    logger = logger
)

# links = bars_scraper_pipeline.get_apartment_links()
# for link in links:
#     bars_scraper_pipeline.scrape_apartment(link)
#     print("Scraped", link)

Creating the file: ../data/bars_apartments.csv


ConnectionError: HTTPSConnectionPool(host='bars.am', port=443): Max retries exceeded with url: /en/properties/standard/apartment (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x11d9f1a30>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [13]:
from ConcreteScrapers.GlobalScrapingPipeline import GlobalScrapingPipeline
global_pipeline = GlobalScrapingPipeline([
    my_realty_scraper_pipeline,
    bars_scraper_pipeline
])
global_pipeline.run()

# GeoService

In [17]:
from Services import GeoFeaturesExtractor
geoService = GeoFeaturesExtractor(["supermarket", "restaurant", "store", "hospital", "subway_station", "school", "beauty_salon"], 1000)
lat, lng = geoService.location("Yerevan/Erebuni/Khorenatsi Street (ER.)")
amenities = geoService.find_nearby_places(f"{lat},{lng}")
print(amenities)

[Amenity: store, Latitude: 40.1503659, Longitude: 44.5231357, Amenity: restaurant, Latitude: 40.1416798, Longitude: 44.5222183, Amenity: school, Latitude: 40.14604459999999, Longitude: 44.5233545, Amenity: school, Latitude: 40.140908, Longitude: 44.5203911, Amenity: beauty_salon, Latitude: 40.1456645, Longitude: 44.52097089999999, Amenity: supermarket, Latitude: 40.1421264, Longitude: 44.5211657, Amenity: store, Latitude: 40.1421264, Longitude: 44.5211657, Amenity: store, Latitude: 40.1423248, Longitude: 44.5224935]


In [18]:
from ConcreteScrapers.Bars.BarsApartmentScraper import BarsApartmentScraper
scraper = BarsApartmentScraper("https://bars.am/en/properties/standard/apartment/53896")
scraper.scrape()
scraper.values()

{'source': 'bars',
 'id': '53896',
 'price': 75000,
 'facilities': ['Building security',
  'Heating system',
  'Open balcony',
  'Washing machine',
  'Kitchen stove',
  'Fridge',
  'Built-in furniture',
  'Kitchen furniture',
  'Transport availability'],
 'location': 'Yerevan/Erebuni/Khorenatsi Street (ER.)',
 'area': 78.0,
 'rooms': 3,
 'floor': 2,
 'storeys': 4,
 'bedrooms': 2,
 'bathrooms': 1,
 'ceiling_height': 3.2,
 'building_type': 'Stalinka c/o',
 'condition': 'Old major overhaul'}